# CNN 지도학습

체스 데이터로 CNN 모델을 지도학습합니다.

이 모델은 이후 강화학습의 정책 신경망으로 사용됩니다.

## 모델 구조
- **입력**: (18, 8, 8) 체스 보드 상태
- **Policy Head**: 4096개 액션에 대한 확률 분포
- **Value Head**: 포지션 평가값 (-1 ~ 1)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from pathlib import Path
from tqdm import tqdm

from parquet_dataset import (
    ParquetChessDataset,
    get_parquet_file_info, 
    split_files_by_ratio
)

torch.manual_seed(42)
np.random.seed(42)

if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    device = torch.device("cuda")
    print(f"✅ CUDA 사용 가능!")
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA 버전: {torch.version.cuda}")
    print(f"   PyTorch 버전: {torch.__version__}")
    print(f"   GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    device = torch.device("cpu")
    print(f"⚠️  CUDA 사용 불가 - CPU 사용")
    print(f"   PyTorch 버전: {torch.__version__}")

print(f"\n사용 장치: {device}")

✅ CUDA 사용 가능!
   GPU: NVIDIA GeForce RTX 5060 Ti
   CUDA 버전: 13.1
   PyTorch 버전: 2.10.0a0+b4e4ee81d3.nv25.12
   GPU 메모리: 15.93 GB

사용 장치: cuda


## 모델 정의

In [2]:
class ChessCNN(nn.Module):
    """
    체스 CNN 모델
    
    Policy Head와 Value Head를 가진 구조로,
    지도학습 후 강화학습의 정책 신경망으로 사용됩니다.
    """
    
    def __init__(self, num_channels=256):
        super().__init__()
        
        # 입력: (batch, 18, 8, 8)
        # 공통 CNN 백본
        self.conv_layers = nn.Sequential(
            # 첫 번째 블록
            nn.Conv2d(18, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            # 두 번째 블록
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            # 세 번째 블록
            nn.Conv2d(128, num_channels, 3, padding=1),
            nn.BatchNorm2d(num_channels),
            nn.ReLU(),
        )
        
        # Policy Head (4096개 액션)
        self.policy_head = nn.Sequential(
            nn.Conv2d(num_channels, 32, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * 8 * 8, 4096),
        )
        
        # Value Head (1개 출력)
        self.value_head = nn.Sequential(
            nn.Conv2d(num_channels, 32, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * 8 * 8, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Tanh()
        )
    
    def forward(self, x, mask=None):
        """
        Args:
            x: (batch, 18, 8, 8) 입력 텐서
            mask: (batch, 4096) 합법 수 마스크 (선택사항)
        
        Returns:
            policy_logits: (batch, 4096) 정책 로짓
            value: (batch, 1) 가치 예측
        """
        # 공통 백본
        features = self.conv_layers(x)
        
        # Policy Head
        policy_logits = self.policy_head(features)
        
        # Mask 적용 (불법 수 제거)
        if mask is not None:
            illegal_mask = mask < 0.5
            policy_logits = policy_logits.masked_fill(illegal_mask, float('-inf'))
        
        # Value Head
        value = self.value_head(features)
        
        return policy_logits, value


model = ChessCNN(num_channels=256).to(device)

# model = torch.compile(model, mode='default')

print(f"모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")
print(f"학습 가능 파라미터 수: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

sample_input = torch.randn(2, 18, 8, 8).to(device)
sample_mask = torch.ones(2, 4096).to(device)
policy_logits, value = model(sample_input, sample_mask)
print(f"\n입력 shape: {sample_input.shape}")
print(f"Policy logits shape: {policy_logits.shape}")
print(f"Value shape: {value.shape}")

모델 파라미터 수: 9,314,433
학습 가능 파라미터 수: 9,314,433

입력 shape: torch.Size([2, 18, 8, 8])
Policy logits shape: torch.Size([2, 4096])
Value shape: torch.Size([2, 1])


## 데이터셋 설정

In [ ]:
PARQUET_DIR = "data/parquet"
TRAIN_RATIO = 0.9

parquet_files, file_lengths = get_parquet_file_info(PARQUET_DIR)

# train_lengths, val_lengths도 함께 반환받아 중복 스캔 방지
train_files, val_files, train_lengths, val_lengths, train_samples, val_samples = split_files_by_ratio(
    parquet_files, 
    file_lengths, 
    train_ratio=TRAIN_RATIO,
    shuffle=False
)

# =============================================================================
# 성능 최적화 설정
# =============================================================================
# GPU 메모리(RTX 2060 6GB) 기준 배치 크기 설정
# - 기존 100 → 512 (AMP 사용 시 더 큰 배치도 가능)
# - 배치가 커지면 학습률도 비례해서 조정 필요 (Cell 7에서 처리)
BASE_BATCH_SIZE = 100  # 기준 배치 크기 (학습률 스케일링용)
BATCH_SIZE = 512       # 실제 사용할 배치 크기

print("\n학습 데이터셋 생성 중...")
train_dataset = ParquetChessDataset(
    parquet_files=train_files,
    file_lengths=train_lengths,  # 파일 길이 정보 전달하여 중복 스캔 제거
    shuffle_files=True,  # 에폭마다 파일 순서 셔플 (학습 다양성 확보)
    seed=42,
    batch_size=BATCH_SIZE  # 배치 단위 yield로 데이터 로딩 병목 해소
)

print("\n검증 데이터셋 생성 중...")
val_dataset = ParquetChessDataset(
    parquet_files=val_files,
    file_lengths=val_lengths,  # 파일 길이 정보 전달하여 중복 스캔 제거
    shuffle_files=False,  # 검증은 파일 순서 고정
    seed=42,
    batch_size=BATCH_SIZE  # 배치 단위 yield로 데이터 로딩 병목 해소
)

print(f"\n최종 데이터셋 크기:")
print(f"  학습 데이터: {train_dataset.estimated_length:,} (스트리밍)")
print(f"  검증 데이터: {val_dataset.estimated_length:,} (스트리밍)")

# 멀티 워커 설정 (GPU 대기 시간 최소화)
# - Windows에서는 spawn 방식이라 오버헤드가 있음, 2~4 권장
# - Linux/Docker에서는 4~8도 가능
NUM_WORKERS = 4
PREFETCH_FACTOR = 4  # 워커당 미리 로드할 배치 수

if torch.cuda.is_available():
    print(f"CUDA 사용 중")
    print(f"  배치 크기: {BATCH_SIZE} (기존 {BASE_BATCH_SIZE}에서 {BATCH_SIZE/BASE_BATCH_SIZE:.1f}x 증가)")
    print(f"  워커 수: {NUM_WORKERS}, 프리패치: {PREFETCH_FACTOR}")

# 에폭당 예상 이터레이션 수 (tqdm 진행률 표시용)
TRAIN_STEPS_PER_EPOCH = train_dataset.estimated_length // BATCH_SIZE
VAL_STEPS_PER_EPOCH = val_dataset.estimated_length // BATCH_SIZE
print(f"\n에폭당 이터레이션:")
print(f"  학습: {TRAIN_STEPS_PER_EPOCH:,} steps")
print(f"  검증: {VAL_STEPS_PER_EPOCH:,} steps")

train_loader = DataLoader(
    train_dataset,
    batch_size=None,  # Dataset이 이미 배치를 반환하므로 None
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=torch.cuda.is_available(),
    persistent_workers=NUM_WORKERS > 0,
    prefetch_factor=PREFETCH_FACTOR if NUM_WORKERS > 0 else None,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=None,  # Dataset이 이미 배치를 반환하므로 None
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=torch.cuda.is_available(),
    persistent_workers=NUM_WORKERS > 0,
    prefetch_factor=PREFETCH_FACTOR if NUM_WORKERS > 0 else None,
)

print("\n첫 배치 로딩 중...")
for states, policies, masks, values in train_loader:
    print(f"\n배치 shape 확인:")
    print(f"  States: {states.shape}")
    print(f"  Policies: {policies.shape}")
    print(f"  Masks: {masks.shape}")
    print(f"  Values: {values.shape}")
    break

print("\n데이터 로딩 준비 완료!")

Parquet 파일 정보 수집 중... (449개 파일)


파일 스캔: 100%|██████████| 449/449 [00:00<00:00, 471.20it/s]

총 파일 수: 449, 총 샘플 수: 22,450,000

파일 단위 분할 완료:
  Train: 405개 파일, 20,250,000개 샘플 (90.2%)
  Val: 44개 파일, 2,200,000개 샘플 (9.8%)

학습 데이터셋 생성 중...
파일 길이 정보 사용 (스캔 생략, 405개 파일)
총 샘플 수: 20,250,000
모드: 파일 순서 셔플, 배치 크기: 512

검증 데이터셋 생성 중...
파일 길이 정보 사용 (스캔 생략, 44개 파일)
총 샘플 수: 2,200,000
모드: 순차 읽기, 배치 크기: 512

최종 데이터셋 크기:
  학습 데이터: 20,250,000 (스트리밍)
  검증 데이터: 2,200,000 (스트리밍)
CUDA 사용 중
  배치 크기: 512 (기존 100에서 5.1x 증가)
  워커 수: 4, 프리패치: 4

에폭당 이터레이션:
  학습: 39,550 steps
  검증: 4,296 steps

첫 배치 로딩 중...


ValueError: too many values to unpack (expected 4)

## 학습 설정

In [ ]:
# =============================================================================
# 학습 하이퍼파라미터
# =============================================================================
BASE_LEARNING_RATE = 0.001  # 기준 배치(100)에서의 학습률

# 선형 스케일링: 배치가 N배 커지면 학습률도 N배
# (단, 너무 크면 발산할 수 있어서 상한 설정)
LR_SCALE = min(BATCH_SIZE / BASE_BATCH_SIZE, 8.0)  # 최대 8배
LEARNING_RATE = BASE_LEARNING_RATE * LR_SCALE

NUM_EPOCHS = 10
POLICY_WEIGHT = 1.0
VALUE_WEIGHT = 1.0

# =============================================================================
# AMP (Automatic Mixed Precision) 설정
# =============================================================================
# - GPU 연산을 float16으로 수행해 속도 향상 + 메모리 절약
# - GradScaler로 gradient underflow 방지
USE_AMP = torch.cuda.is_available()
scaler = torch.amp.GradScaler('cuda', enabled=USE_AMP)

policy_loss_fn = nn.CrossEntropyLoss()
value_loss_fn = nn.MSELoss()

optimizer = optim.Adam(
    model.parameters(), 
    lr=LEARNING_RATE,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0.0,
    amsgrad=False
)

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=2
)

# 모델 저장 경로
MODEL_DIR = Path("models")
MODEL_DIR.mkdir(exist_ok=True)
BEST_MODEL_PATH = MODEL_DIR / "best_chess_cnn.pth"
LAST_MODEL_PATH = MODEL_DIR / "last_chess_cnn.pth"

LOG_DIR = MODEL_DIR / "tensorboard"
writer = SummaryWriter(log_dir=LOG_DIR)

print(f"학습 설정:")
print(f"  Base LR: {BASE_LEARNING_RATE} → Scaled LR: {LEARNING_RATE:.6f} ({LR_SCALE:.1f}x)")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Policy Weight: {POLICY_WEIGHT}")
print(f"  Value Weight: {VALUE_WEIGHT}")
print(f"  AMP (Mixed Precision): {'활성화' if USE_AMP else '비활성화'}")
print(f"\nTensorBoard 로그 디렉토리: {LOG_DIR}")
print(f"TensorBoard 실행: tensorboard --logdir={LOG_DIR}")
print(f"브라우저에서 http://localhost:6006 접속")

학습 설정:
  Base LR: 0.001 → Scaled LR: 0.005120 (5.1x)
  Epochs: 10
  Batch Size: 512
  Policy Weight: 1.0
  Value Weight: 1.0
  AMP (Mixed Precision): 활성화

TensorBoard 로그 디렉토리: models/tensorboard
TensorBoard 실행: tensorboard --logdir=models/tensorboard
브라우저에서 http://localhost:6006 접속


## 학습 함수

In [ ]:
def train_epoch(model, dataloader, optimizer, policy_loss_fn, value_loss_fn, 
                policy_weight, value_weight, device, scaler=None, use_amp=False,
                total_steps=None):
    """
    한 에폭 학습 (AMP 지원)
    
    Args:
        scaler: torch.amp.GradScaler (AMP 사용 시)
        use_amp: AMP 활성화 여부
        total_steps: tqdm 진행률 표시용 총 스텝 수
    """
    model.train()
    total_policy_loss = 0.0
    total_value_loss = 0.0
    total_loss = 0.0
    num_batches = 0
    
    # CUDA 사용 시 비동기 전송
    non_blocking = torch.cuda.is_available()
    
    pbar = tqdm(dataloader, desc="학습 중", total=total_steps)
    for states, policies, masks, values in pbar:
        # CUDA 사용 시 비동기 전송으로 속도 향상
        states = states.to(device, non_blocking=non_blocking)
        policies = policies.to(device, non_blocking=non_blocking)
        masks = masks.to(device, non_blocking=non_blocking)
        values = values.to(device, non_blocking=non_blocking)
        
        optimizer.zero_grad(set_to_none=True)
        
        # AMP: autocast로 forward/loss를 float16으로 수행
        with torch.amp.autocast('cuda', enabled=use_amp):
            policy_logits, value_pred = model(states, masks)
            policy_loss = policy_loss_fn(policy_logits, policies)
            value_loss = value_loss_fn(value_pred.squeeze(), values)
            loss = policy_weight * policy_loss + value_weight * value_loss
        
        # AMP: scaler로 backward (gradient scaling)
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=2)
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=2)
            optimizer.step()
        
        # 통계
        total_policy_loss += policy_loss.item()
        total_value_loss += value_loss.item()
        total_loss += loss.item()
        num_batches += 1
        
        # 진행 상황 업데이트
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'policy': f'{policy_loss.item():.4f}',
            'value': f'{value_loss.item():.4f}'
        })
    
    return {
        'policy_loss': total_policy_loss / num_batches,
        'value_loss': total_value_loss / num_batches,
        'total_loss': total_loss / num_batches
    }


def validate(model, dataloader, policy_loss_fn, value_loss_fn, 
             policy_weight, value_weight, device, use_amp=False, total_steps=None):
    """
    검증 (AMP 지원)
    
    Args:
        use_amp: AMP 활성화 여부
        total_steps: tqdm 진행률 표시용 총 스텝 수
    """
    model.eval()
    total_policy_loss = 0.0
    total_value_loss = 0.0
    total_loss = 0.0
    num_batches = 0
    
    correct_predictions = 0
    total_predictions = 0
    
    # CUDA 사용 시 비동기 전송
    non_blocking = torch.cuda.is_available()
    
    with torch.no_grad():
        for states, policies, masks, values in tqdm(dataloader, desc="검증 중", total=total_steps):
            states = states.to(device, non_blocking=non_blocking)
            policies = policies.to(device, non_blocking=non_blocking)
            masks = masks.to(device, non_blocking=non_blocking)
            values = values.to(device, non_blocking=non_blocking)
            
            # AMP: autocast로 forward를 float16으로 수행
            with torch.amp.autocast('cuda', enabled=use_amp):
                policy_logits, value_pred = model(states, masks)
                policy_loss = policy_loss_fn(policy_logits, policies)
                value_loss = value_loss_fn(value_pred.squeeze(), values)
                loss = policy_weight * policy_loss + value_weight * value_loss
            
            # 정확도 계산
            pred_policies = policy_logits.argmax(dim=1)
            correct_predictions += (pred_policies == policies).sum().item()
            total_predictions += policies.size(0)
            
            # 통계
            total_policy_loss += policy_loss.item()
            total_value_loss += value_loss.item()
            total_loss += loss.item()
            num_batches += 1
    
    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0.0
    
    return {
        'policy_loss': total_policy_loss / num_batches,
        'value_loss': total_value_loss / num_batches,
        'total_loss': total_loss / num_batches,
        'accuracy': accuracy
    }

## 학습 실행

In [ ]:
# 학습 히스토리
train_history = {
    'policy_loss': [],
    'value_loss': [],
    'total_loss': []
}

val_history = {
    'policy_loss': [],
    'value_loss': [],
    'total_loss': [],
    'accuracy': []
}

best_val_loss = float('inf')

print("=" * 60)
print("학습 시작")
print("=" * 60)

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
    print("-" * 60)
    
    # 에폭 설정 (파일 순서 셔플에 사용)
    train_dataset.set_epoch(epoch)
    
    # 학습
    train_metrics = train_epoch(
        model, train_loader, optimizer, policy_loss_fn, value_loss_fn,
        POLICY_WEIGHT, VALUE_WEIGHT, device,
        scaler=scaler, use_amp=USE_AMP, total_steps=TRAIN_STEPS_PER_EPOCH
    )
    
    # 검증
    val_metrics = validate(
        model, val_loader, policy_loss_fn, value_loss_fn,
        POLICY_WEIGHT, VALUE_WEIGHT, device,
        use_amp=USE_AMP, total_steps=VAL_STEPS_PER_EPOCH
    )
    
    # 히스토리 저장
    train_history['policy_loss'].append(train_metrics['policy_loss'])
    train_history['value_loss'].append(train_metrics['value_loss'])
    train_history['total_loss'].append(train_metrics['total_loss'])
    
    val_history['policy_loss'].append(val_metrics['policy_loss'])
    val_history['value_loss'].append(val_metrics['value_loss'])
    val_history['total_loss'].append(val_metrics['total_loss'])
    val_history['accuracy'].append(val_metrics['accuracy'])
    
    # 결과 출력
    print(f"\n학습 결과:")
    print(f"  Policy Loss: {train_metrics['policy_loss']:.4f}")
    print(f"  Value Loss: {train_metrics['value_loss']:.4f}")
    print(f"  Total Loss: {train_metrics['total_loss']:.4f}")
    
    print(f"\n검증 결과:")
    print(f"  Policy Loss: {val_metrics['policy_loss']:.4f}")
    print(f"  Value Loss: {val_metrics['value_loss']:.4f}")
    print(f"  Total Loss: {val_metrics['total_loss']:.4f}")
    print(f"  Accuracy: {val_metrics['accuracy']:.4f} ({val_metrics['accuracy']*100:.2f}%)")
    
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(val_metrics['total_loss'])
    new_lr = optimizer.param_groups[0]['lr']
    if old_lr != new_lr:
        print(f"\n📉 학습률 감소: {old_lr:.6f} -> {new_lr:.6f}")
    else:
        print(f"  현재 학습률: {new_lr:.6f}")
    
    if val_metrics['total_loss'] < best_val_loss:
        best_val_loss = val_metrics['total_loss']
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_loss': val_metrics['total_loss'],
            'val_accuracy': val_metrics['accuracy'],
            'train_loss': train_metrics['total_loss'],
            'learning_rate': optimizer.param_groups[0]['lr'],
            'model_config': {'num_channels': 256},
        }
        torch.save(checkpoint, BEST_MODEL_PATH)
        print(f"\n✅ 최고 모델 저장! (Loss: {best_val_loss:.4f})")
    
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'val_loss': val_metrics['total_loss'],
        'val_accuracy': val_metrics['accuracy'],
        'train_loss': train_metrics['total_loss'],
        'learning_rate': optimizer.param_groups[0]['lr'],
        'model_config': {'num_channels': 256},
    }
    torch.save(checkpoint, LAST_MODEL_PATH)

print("\n" + "=" * 60)
print("학습 완료!")
print("=" * 60)
print(f"최고 검증 Loss: {best_val_loss:.4f}")
print(f"최고 모델: {BEST_MODEL_PATH}")

학습 시작

Epoch 1/10
------------------------------------------------------------


학습 중:   3%|▎         | 1280/39550 [03:13<1:36:22,  6.62it/s, loss=4.0304, policy=2.2609, value=1.7695]
Exception in thread Thread-5 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1010, in run


KeyboardInterrupt: 

    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 52, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/pin_memory.py", line 28, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/resource_sharer.py", line 86, in get_connection
 

## 학습 곡선 시각화

In [ ]:
import matplotlib.pyplot as plt

# Loss 곡선
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Total Loss
axes[0, 0].plot(train_history['total_loss'], label='Train', marker='o')
axes[0, 0].plot(val_history['total_loss'], label='Validation', marker='s')
axes[0, 0].set_title('Total Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Policy Loss
axes[0, 1].plot(train_history['policy_loss'], label='Train', marker='o')
axes[0, 1].plot(val_history['policy_loss'], label='Validation', marker='s')
axes[0, 1].set_title('Policy Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Value Loss
axes[1, 0].plot(train_history['value_loss'], label='Train', marker='o')
axes[1, 0].plot(val_history['value_loss'], label='Validation', marker='s')
axes[1, 0].set_title('Value Loss')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Accuracy
axes[1, 1].plot(val_history['accuracy'], label='Validation', marker='s', color='green')
axes[1, 1].set_title('Policy Accuracy')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig(MODEL_DIR / 'training_curves.png', dpi=150)
plt.show()

print("학습 곡선 저장 완료:", MODEL_DIR / 'training_curves.png')

## 모델 평가

In [ ]:
checkpoint = torch.load(BEST_MODEL_PATH, map_location=device, weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"로드된 모델 정보:")
print(f"  Epoch: {checkpoint['epoch']}")
print(f"  Validation Loss: {checkpoint['val_loss']:.4f}")
print(f"  Validation Accuracy: {checkpoint['val_accuracy']:.4f} ({checkpoint['val_accuracy']*100:.2f}%)")
if 'learning_rate' in checkpoint:
    print(f"  Learning Rate: {checkpoint['learning_rate']:.6f}")
if 'train_loss' in checkpoint:
    print(f"  Train Loss: {checkpoint['train_loss']:.4f}")

# 최종 검증
final_metrics = validate(
    model, val_loader, policy_loss_fn, value_loss_fn,
    POLICY_WEIGHT, VALUE_WEIGHT, device,
    use_amp=USE_AMP, total_steps=VAL_STEPS_PER_EPOCH
)

print(f"\n최종 검증 결과:")
print(f"  Policy Loss: {final_metrics['policy_loss']:.4f}")
print(f"  Value Loss: {final_metrics['value_loss']:.4f}")
print(f"  Total Loss: {final_metrics['total_loss']:.4f}")
print(f"  Accuracy: {final_metrics['accuracy']:.4f} ({final_metrics['accuracy']*100:.2f}%)")

## 샘플 예측 확인

In [ ]:
# 몇 개 샘플로 예측 확인
model.eval()
non_blocking = torch.cuda.is_available()

with torch.no_grad():
    for states, policies, masks, values in val_loader:
        states = states.to(device, non_blocking=non_blocking)
        masks = masks.to(device, non_blocking=non_blocking)
        
        policy_logits, value_pred = model(states, masks)
        policy_probs = torch.softmax(policy_logits, dim=1)
        pred_policies = policy_logits.argmax(dim=1)
        
        print("샘플 예측:")
        for i in range(min(5, len(states))):
            print(f"\n샘플 {i+1}:")
            print(f"  실제 Policy: {policies[i].item()}")
            print(f"  예측 Policy: {pred_policies[i].item()}")
            print(f"  정확도: {'✅' if pred_policies[i] == policies[i] else '❌'}")
            print(f"  실제 Value: {values[i].item():.4f}")
            print(f"  예측 Value: {value_pred[i].item():.4f}")
            print(f"  예측 확률 (top-1): {policy_probs[i].max().item():.4f}")
        
        break

## 모델 저장 (강화학습용)

In [ ]:
# 강화학습용 모델 저장 (state_dict만)
RL_MODEL_PATH = MODEL_DIR / "chess_cnn_rl.pth"
torch.save(model.state_dict(), RL_MODEL_PATH)

print(f"강화학습용 모델 저장 완료: {RL_MODEL_PATH}")
print(f"\n강화학습에서 사용 방법:")
print(f"  model = ChessCNN(num_channels=256)")
print(f"  model.load_state_dict(torch.load('{RL_MODEL_PATH}'))")
print(f"  model.eval()")